# Исследование надёги-безнадёги заёмщиков

 Перед нами стоит задача по определению надёжности заёмщика кредитных средств у банка. Нулевой гипотезой будет отсутствие корреляции между семейным положением и количеством детей в анализе надёжности заёмщика, пока не будет доказано обратное. Входные данные предоставлены банком - это статистика о платёжеспособности клинетов. 

Описание данных из статистики: 
children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

In [1]:
# Шаг 1. Открываем таблицу, выводим её на экран, методами .info() и describe () выводим информацию о массиве
import pandas as pd
import numpy as np

data = pd.read_csv('/Users/zhorzheta/Downloads/data.csv')
data.info()
data.describe()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


#### Аномалии, которые удалось обнаружить на данном этапе:
- разный регистр в столбце образования (привести к общему регистру) + разный регистр может быть в столбцах семейного положения, типе занятости и цели получения кредита, приведем их тоже к общему регистру
- стаж и количество детей в отрицательных значениях (перевести в положительные)
- стаж в днях, что неудобно для понимания масштаба времени (конвертировать в года)
- доход и стаж - не целочисленные числа (превратить в целые, float 64 преобразуем в int64)
- столбец "family_status_id" повторяет информацией столбец "family": говорит о семейном положении, прибавляя ещё и индексацию от 0 до 4, что можно считать избыточностью (убрать столбец 'family_status_id'); то же самое касается и столбца 'education_id' с индексацией (убрать столбец 'education_id')


In [2]:
# Шаг 3. Поработаем с пропусками, найдем их
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество пропусков в днях стажа и ежемесячном доходе - совпадают. Если не было работы, значит - не было и компенсации, либо это случай неофициальной рабочей деятельности. В любом случае, они отсутствуют. Конечно, можно заполнить пропуски средним арифметическим значением или медианным, но тогда получится как в шутке про Билла Гейтса, который зашел в бар и все сидящие в баре стали миллионерами. Для того, чтобы не нарушить корректность данных и их анализ - вставим нулевые значения в пропусках. Да, будут потеряны 2 174 записи (~ 10% массива), однако весь остальной массив не будет похерен недостоверными данными. Плюс, люди, не имеющие стабильного дохода и/или работы это априори ненадёжный заёмщик банковских денег.  

In [3]:
data['days_employed'] = data['days_employed'].fillna(0)
data['total_income'] = data['total_income'].fillna(0)

In [4]:
# Делаем проверку на пропуски
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [5]:
# Делаем проверку на дубли
data.duplicated().sum()

54

54 дубля - откуда они появились, один вопрос, другой - это не критичное количество, которое можно удалить, что и сделаем, сбросив при  этом индекс. Проверим ушли ли все дубли. 

In [6]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

Обработаем теперь столбец со стажем, переведем дни в года, и сохраним это значение в новом столбце + отрицательные числа переведем в положительные значения + сделаем целочисленные показатели.

In [7]:
data['years_employed'] = data['days_employed'] / 365

In [8]:
data['years_employed'].value_counts()

 0.000000     2120
-16.233391       1
-2.391293        1
-8.823936        1
-6.962928        1
              ... 
-4.060155        1
-0.514288        1
-3.140524        1
-4.560083        1
-6.327623        1
Name: years_employed, Length: 19352, dtype: int64

In [9]:
data['years_employed'] = data['years_employed'].astype(int)
data['years_employed'] = data['years_employed'].abs()
data['years_employed'].value_counts()

0     3947
1     2023
2     1878
3     1560
4     1350
      ... 
42       2
44       2
50       1
45       1
48       1
Name: years_employed, Length: 249, dtype: int64

Обнаружены интересные аномалии, люди работают не одну тысячу лет, что делать с этим на данном этапе исследования - непонятно (скорее всего, следует обратиться к тем, кто выдавал массив с запросом на проверку данных аномалий), но для подтверждения или опровержения нулевой гипотезы о корреляции семейного статуса и наличия детей нам этот столбец не понадобится. Поехали вычищать другие столбцы.

In [10]:
# Приводим к нижнему регистру все столбцы с буквами, исключая пол
data['family_status'] = data['family_status'].str.lower()
data['education'] = data['education'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['gender'].str.isupper()

0        True
1        True
2        True
3        True
4        True
         ... 
21466    True
21467    True
21468    True
21469    True
21470    True
Name: gender, Length: 21471, dtype: bool

Поработаем со значениями в столбце гендера, попробуем там найти аномалии.

In [11]:
data['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

Так как значений "XNA" только одно, поменяем его на мужчину, глобально это не повлияет на выкладку.

In [12]:
data['gender'] = data['gender'].replace('XNA', 'M')

In [13]:
data['gender'].value_counts()

F    14189
M     7282
Name: gender, dtype: int64

Теперь разберемся со столбцом о детях, отрицательные превратим в положительные показатели сначала.

In [14]:
data['children'] = data['children'].abs()

In [15]:
data['children'].value_counts()

0     14107
1      4856
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Видим, что 76 клиентов - это семьи с 20 детьми. Для такого большого количество детей - 76 семей это очень много. Скорее всего, в массив загрузили лишний ноль, и у этих представителей по 2 детей, но это предположение - пальцем в небо, а столбец важен для проверки нулевой гипотезы, лучше удалить, чтобы анализ был корректен.

In [16]:
data = data[(data['children'] >= 0) & (data['children'] <= 5)]
data.sort_values('children')['children'].value_counts().sort_index()

0    14107
1     4856
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Превратим доход и стаж в целочисленные значения.

In [49]:
timedian = data['total_income'].median()

In [50]:
def categorize_total_income(value):
    if value < 50_000:
        return 'низкий'
    elif value <= timedian:
        return 'средний'
    elif value < 350_000:
        return 'выше среднего'
    else:
        return 'высокий'

In [51]:
data['total_income_category'] = data['total_income'].apply(categorize_total_income)
data['total_income_category']

0        выше среднего
1              средний
2        выше среднего
3        выше среднего
4        выше среднего
             ...      
21466    выше среднего
21467    выше среднего
21468          средний
21469    выше среднего
21470          средний
Name: total_income_category, Length: 21395, dtype: object

Разберёмся со столбцами, что являются избыточными, то есть "family_status_id" и "education_id".

In [18]:
# Посмотрим, действительно ли индексация совпадает с количеством возможных вариантов семейного статуса, а потом и образования
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [19]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

In [20]:
# Все совпадает, удалим столбец "family_status_id"
del data['family_status_id']

In [21]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [22]:
data['education_id'].unique()

array([0, 1, 2, 3, 4])

In [23]:
# То же самое сделаем и со столбцом "education_id"
del data['education_id']

In [24]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [68]:
# Выделим леммы в значениях столбца с целями получения кредита
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
 
 
 
def lemm(row): 
    lemmas = ' '.join(m.lemmatize(row))
    return lemmas
 
data['purpose_edit'] = data['purpose'].apply(lemm) 
 
from collections import Counter 
purpose_list = data['purpose_edit'].tolist() 
 
Counter(' '.join(purpose_list).split())

Counter({'покупка': 5881,
         'жилье': 4446,
         'приобретение': 461,
         'автомобиль': 4292,
         'дополнительный': 903,
         'образование': 3999,
         'сыграть': 765,
         'свадьба': 2326,
         'операция': 2594,
         'с': 2905,
         'на': 2217,
         'проведение': 769,
         'для': 1288,
         'семья': 637,
         'недвижимость': 6332,
         'коммерческий': 1305,
         'жилой': 1228,
         'строительство': 1872,
         'собственный': 629,
         'подержать': 472,
         'свой': 2227,
         'со': 626,
         'заниматься': 904,
         'сделка': 937,
         'подержанный': 482,
         'получение': 1312,
         'высокий': 1368,
         'профильный': 434,
         'сдача': 651,
         'ремонт': 605})

Шаг 3. Ответы на поставленные вопросы

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [43]:
pd.pivot_table(data, index='children', values='debt')

,debt
children,
0,0.075353
1,0.091639
2,0.094542
3,0.081818
4,0.097561
5,0.000000


Лучший показатель у бездетных клинетов: ~7.5%
Далее идут клиенты с 3 детьми: ~8.1%
Лучше всего выплачивают клинеты с 1, 2 и 4 детьми: ~9.1%, ~9.4%, ~9.7% соответсвенно 
Итак, одна часть нулевой гипотезы, выдвинутой в начале исследования не подтвердилась, а именно количество детей влияет на надёжность заёмщика. Кто-то может сказать, что 1.5%-2% небольшая разница в показателе, однако, эти проценты легко превращаются миллионы денежных единиц. 

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [44]:
pd.pivot_table(data, index='family_status', values='debt')

,debt
family_status,
в разводе,0.070411
вдовец / вдова,0.065969
гражданский брак,0.092749
женат / замужем,0.075478
не женат / не замужем,0.097465


Лучший показатель по возврату кредита у вдовцов/вдов: 6.5%
Следующими идут разведённые и семейные с разницей в ~ половину процента: 7%, 7.5% соответственно
За ними следуют пары в гражданском браке и не женатые/не замужние с такой же разницей в ~ половину процента: 9.2%, 9.7% соответсвенно. 
Чуть-чуть осветим терминологию: гражданский браком у нас некорректно, принимая во внимание юридическую терминологию, называют пары, не состоящие в официальном зарегистрированном браке (т.е. гражданском), поэтому их и не женатых/не замужних было бы вернее объединить в одну категорию. 
Теперь ко второй части нулевой гипотезы, что вновь не подтверждается - семейный статус влияет на возврат заёмных средств, максимальная разница в возврате составляет 3.2%. Пары, оформившие свои отношения (или их прекратившие официально) надёжнее для банка, чем люди, что холосты/не замужем.

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [52]:
pd.pivot_table(data, index='total_income_category', values='debt')

,debt
total_income_category,
высокий,0.064252
выше среднего,0.082004
низкий,0.077729
средний,0.082532


Лучший возврат по заёмным средствам уходит клиентам с высоким уровнем дохода: 6.4%
Далее идет клиенты с низким ежемесячным доходом: 7.7%
Равные показатели демонистрируют клинеты с доходом средним и вышего среднего: 8.2%
Как можно интерпретировать результат: люди с высоким доходом являются привлекательными клиентами для банка, по логике убывания далее должны бы следовать люди со средним/выше среднего дохода, однако люди с низким доходом идут на втором месте по надёжности.

4. Как разные цели кредита влияют на его возврат в срок?

In [72]:
data_pivot_purpose = data.pivot_table(index='purpose_edit', columns='debt', values='purpose', aggfunc='count')
# Процент должников
data_pivot_purpose['ratio %'] = (data_pivot_purpose[1] / data_pivot_purpose[0]) * 100
data_pivot_purpose['ratio %'] = data_pivot_purpose['ratio %'].astype('int')
# Cумма клиентов каждой категории
data_pivot_purpose['sum'] = data_pivot_purpose[1] + data_pivot_purpose[0]

data_pivot_purpose.loc[:, ['ratio %', 'sum']]

debt,ratio %,sum
purpose_edit,,
автомобиль \n,9,970
высокий образование \n,9,447
дополнительный образование \n,9,457
жилье \n,7,642
заниматься высокий образование \n,9,496
заниматься образование \n,10,408
на покупка автомобиль \n,10,471
на покупка подержать автомобиль \n,8,472
на покупка свой автомобиль \n,10,505


Лучше всего отдают кредиты, взятые для приобретения или другой операции с недвижимости (124 пункта(не уверена, что я так могу складывать ratio и потом говорить о пунктах, но продолжим) ), далее идут автомобильные кредиты (80 пунктов) и с разницей в один пункт - образование (79 пунктов). Существенно хуже люди отдают деньги, взятые для реализации свадебных торжеств (24 пункта, что составляет лишь 19.3% от возврата по недвижимости).

Вывод:
идеальный портрет клинета выглядит примерно так:
- бездетный /ая
- овдовеший/ая (к сожалению), в браке/разведены
- с высоким или низким ежемесчным доходом (лучше высоким)
Было бы интересно еще проанализировать пол идеального клиента, но кто-то мог бы назвать это сексизом (а кто-то просто статистикой, которая оперирует вне рамок этики). 


"Спасибо, что дочитали до конца!"